In [1]:
!pip install selenium --quiet

In [ ]:
import time

from selenium import webdriver

driver = webdriver.Chrome() 

# Need to download compatible Chrome Driver

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options

from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import pandas as pd
import csv
import re
import time
import os 

In [8]:
url = 'https://www.wsj.com/search?query=SPAC&isToggleOn=true&operator=AND&sort=date-desc&duration=4y&startDate=2018%2F03%2F09&endDate=2022%2F03%2F09&source=wsjie'
# Articles with keyword 'SPAC' over thew past 4 years

#options = webdriver.ChromeOptions()
#options.add_argument("start-maximized");
#options.add_argument("disable-infobars")
#options.add_argument("--disable-extensions")
#chrome_options = Options()
#chrome_options.add_argument("--window-size=1920,1080")
#prefs = {"profile.managed_default_content_settings.images": 2}
#chrome_options.add_experimental_option("prefs", prefs)
#driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
# url_article = requests(url)
driver = webdriver.Chrome()  
driver.get(url)

Sign in manually

In [9]:
article_links = driver.find_elements(By.XPATH,'//*[@id="main"]/div/article/div/div/div/div/h3/a')
article_links[1:10]
# Unfortunately these links change due to dynamic id, so we can't put them in a loop.

[<selenium.webdriver.remote.webelement.WebElement (session="679e2a6d6d3d23deacc6b0171afd1977", element="0fad3612-0866-4259-9edc-585cafc27aa7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="679e2a6d6d3d23deacc6b0171afd1977", element="96eaf5df-3fa1-47d6-956e-15d6d6b1853a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="679e2a6d6d3d23deacc6b0171afd1977", element="d75da8f0-a2ce-4221-ab85-f4186d0936b6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="679e2a6d6d3d23deacc6b0171afd1977", element="69b91633-fc2d-49f6-8a1a-0eaaf17f88d7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="679e2a6d6d3d23deacc6b0171afd1977", element="da7d0c81-7c34-4d6c-b7a5-37f780f151d3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="679e2a6d6d3d23deacc6b0171afd1977", element="d9055f45-4f75-4cb7-87be-4305d63a8f7c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="679e2a6d6d3d23deacc6b0171afd1977", element="4d9a0ab6-4597-4879-8fd8-6e

In [ ]:
headline = []
subheading = []
time_stamp = []
content = []
for i in range(0,20):
    article_links = driver.find_elements(By.XPATH,'//*[@id="main"]/div/article/div/div/div/div/h3/a')
    article_links[i].click()
    WebDriverWait(driver, 40).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="main"]/div/div/div')))
    try:
        headline.append(driver.find_element(By.XPATH,'.//h1[@class="wsj-article-headline"]').text)
    except:
        head_alt = driver.find_element(By.TAG_NAME,'title').get_attribute('textContent')
        headline.append(head_alt)
    try:
        subheading.append(driver.find_element(By.XPATH,'//*[@id="main"]/header/div/div/h2').text)
    except: 
        subheading.append(driver.find_element(By.CLASS_NAME,'bigTop__dek').text)
    time_stamp.append(driver.find_element(By.XPATH,".//time[@class='timestamp article__timestamp flexbox__flex--1']").text)
    content.append(driver.find_element(By.XPATH,'//*[@id="main"]/div/div/div').text)
    driver.back()
    WebDriverWait(driver, 40).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="main"]/div/article/div/div/div/div/h3/a')))

In [ ]:
# Let's save the first 20 before the monster loop. If anything happens, we have back-up
currentFrame = pd.DataFrame({'headline':headline,'subheading':subheading,'timestamp':time_stamp,'content':content})
currentFrame.to_csv('20 articles for 20 hours.csv',index=False)

In [22]:
# if messed up, reload the back-up and reassign the lists

currentFrame= pd.read_csv('20 articles for 20 hours.csv')
headline=currentFrame['headline'].tolist()
subheading = currentFrame['subheading'].tolist()
time_stamp = currentFrame['timestamp'].tolist()
content = currentFrame['content'].tolist()
type(headline)
subheading[0:5]

['Company traps waste gas, uses bacteria to turn it into sustainable chemicals and products for customers including Unilever and Coty',
 'Some estimates put the unit’s projected valuation at above $50 billion',
 'Company is latest in a string of SPAC deals that have performed poorly in the stock market',
 'Hot upstarts that went public via blank-check companies are missing revenue and earnings targets—sometimes months after making those forecasts',
 'Major indexes recover after invasion of Ukraine sends stocks lower and oil above $100 a barrel']

In [ ]:
# Now when we go back to the first page that had a list of 20 articles
# Click next
# and attempt a similar strategy
while True:
    driver.find_elements(By.XPATH,'//a[@class="WSJTheme--button--12LOad_- typography--sans-serif--1WZesAGA WSJTheme--pagination--3MTI0FnK WSJTheme--next--2r7-j2I8 "][@href]')[0].click()
    WebDriverWait(driver, 35).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="main"]/div/article/div/div/div/div/h3/a')))
    
    for i in range(0,20):
        article_links = driver.find_elements(By.XPATH,'//*[@id="main"]/div/article/div/div/div/div/h3/a')
        article_links[i].click()
        WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="wsj-article-wrap"]')))
        try:
            headline.append(driver.find_element(By.XPATH,'.//h1[@class="wsj-article-headline"]').text)
        except:
            head_alt = driver.find_element(By.TAG_NAME,'title').get_attribute('textContent')
            headline.append(head_alt)
        try:
            subheading.append(driver.find_element(By.XPATH,'//*[@id="main"]/header/div/div/h2').text)
        except: 
            try:
                subheading.append(driver.find_element(By.CLASS_NAME,'bigTop__dek').text)
            except:
                subheading.append('')
        time_stamp.append(driver.find_element(By.XPATH,".//time[@class='timestamp article__timestamp flexbox__flex--1']").text)
        content.append(driver.find_element(By.XPATH,'//*[@id="wsj-article-wrap"]').text)
        driver.back()
        WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="main"]/div/article/div/div/div/div/h3/a')))
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//a[@class="WSJTheme--button--12LOad_- typography--sans-serif--1WZesAGA WSJTheme--pagination--3MTI0FnK WSJTheme--next--2r7-j2I8 "][@href]')))


The loop stopped at article 53 because it is a list of articles with no subheader.
Bypass this with try-catch and another try-catch for the except term to catch ohter strange format
Should have try-catch all lists to keep the loop running till the end. It finally stopped at article 354 because 355 had strange format.

In [75]:
dfwsj = pd.DataFrame({'headline':headline,'subheading':subheading,'timestamp':time_stamp,'content':content}).drop_duplicates()
dfwsj.to_excel('final_WSJ_News.xlsx',index = False)